### This file shows the complete process of developing the model from preprocessing to training and finding suitable parameters

## Preprocessing

First, read the csv file and drop unused columns.

The following function drops unused columns, calculates the mean of collinear variables and adds a new column "month" describing in which month the observation was measured.

In [401]:
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def transform_data_median(data):
    def mean_by_height(substring):
        return data.filter(like = substring).filter(like = ".mean").median(axis = 1)
    
    def std_by_height(substring):
        return data.filter(like = substring).filter(like = ".std").median(axis = 1)
    
    data = data.drop(["id", "partlybad"], axis = 1)
    class2 = np.array(["nonevent", "event"])
    data.insert(1, "class2", class2[(data["class4"] != "nonevent").astype(int)])

    new_data = pd.DataFrame()
    new_data[['date', 'class2']] = data[['date', 'class2']]
    new_data["CO.mean"] = mean_by_height("CO")
    new_data["CO.std"] = std_by_height("CO")
    new_data["H2O.mean"] = mean_by_height("H2O")
    new_data["H2O.std"] = std_by_height("H2O")
    new_data["RHIRGA.mean"] = mean_by_height("RHIRGA")
    new_data["RHIRGA.std"] = std_by_height("RHIRGA")
    new_data["NOx.mean"] = mean_by_height("NOx")
    new_data["NOx.std"] = std_by_height("NOx")
    new_data["NET.mean"] = mean_by_height("NET")
    new_data["NET.std"] = std_by_height("NET")
    new_data["NO.mean"] = data.iloc[:,27:39].filter(like = ".mean").median(axis = 1)
    new_data["NO.std"] = data.iloc[:,27:39].filter(like = ".std").median(axis = 1)
    new_data["O3.mean"] = data.iloc[:,51:61].filter(like = ".mean").median(axis = 1)
    new_data["O3.std"] = data.iloc[:, 51:61].filter(like = ".std").median(axis = 1)
    new_data[['Pamb0.mean', 'Pamb0.std', 'PAR.mean', 'PAR.std', 'PTG.mean', 'PTG.std', 'RGlob.mean', 'RGlob.std']] = data[['Pamb0.mean', 'Pamb0.std', 'PAR.mean', 'PAR.std', 'PTG.mean', 'PTG.std', 'RGlob.mean', 'RGlob.std']]
    new_data[['RPAR.mean', 'RPAR.std', 'SO2168.mean', 'SO2168.std', 'SWS.mean', 'SWS.std']] = data[['RPAR.mean', 'RPAR.std', 'SO2168.mean', 'SO2168.std', 'SWS.mean', 'SWS.std']]
    new_data["T.mean"] = data.filter(like = "T").iloc[:,4:].filter(like = ".mean").median(axis = 1)
    new_data["T.std"] = data.filter(like = "T").iloc[:,4:].filter(like = ".std").median(axis = 1)
    new_data[['UV_A.mean', 'UV_A.std', 'UV_B.mean', 'UV_B.std', 'CS.mean', 'CS.std']] = data[['UV_A.mean', 'UV_A.std', 'UV_B.mean', 'UV_B.std', 'CS.mean', 'CS.std']]
    new_data.set_index('date')

    return new_data

# Binary classification

After preprocessing, we can start to develop the model. From previous testing, it was found that logistic regression is the best performer. But before that, we need to scale our data with standard scaler.

In [402]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

train_data = transform_data_median(pd.read_csv("npf_train.csv"))
test_data = transform_data_median(pd.read_csv("npf_test.csv"))

X_train = train_data.iloc[:,2:]
y_train = (train_data.iloc[:,1] == 'event').astype(float)

X_test = test_data.iloc[:,2:]
y_test = (test_data.iloc[:,1] == 'event').astype(float)

#Scale the data
scaler = StandardScaler().fit(pd.concat([X_train, X_test], axis = 0))

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Feature selection
lsvc = LinearSVC(C = 0.1, penalty = 'l1', dual = False).fit(X_train,y_train)
model = SelectFromModel(lsvc, prefit=True)

#Apply the same model to the test data
X_train = model.transform(X_train)
X_test = model.transform(X_test)

### Estimating accuracy for binary classification

To avoid overfitting, we reduce the dimensions with PCA. We will choose number of components with k-fold cross-validation on logistic regression. L2 regularization is used to lower the variance to avoid overfitting.

In [403]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate


clf = LogisticRegression(penalty = 'l1', C = 1/4, solver = "saga", max_iter = 5000)

scores = cross_validate(
    clf, X_train, y_train, cv = 5, scoring = ('accuracy', 'neg_mean_squared_error')
)

est_acc = scores['test_accuracy'].mean()
mean_squared = -scores['test_neg_mean_squared_error'].mean()
print(f'The estimated accuracy of the model is: {est_acc}')
print(f'The estimated MSE is {mean_squared}')

The estimated accuracy of the model is: 0.8792893875642823
The estimated MSE is 0.12071061243571761


# Predictions on the test data

In [404]:
predictions = pd.DataFrame()

predictions["class4"] = clf.fit(X_train, y_train).predict(X_test)
predictions["p"] = clf.fit(X_train, y_train).predict_proba(X_test)[:,1]
predictions["class4"] = predictions["class4"].map({1.0: 'event', 0.0:'nonevent'})
predictions.to_csv("answers.csv", index = False)

predictions

,class4,p
0,event,0.892169
1,nonevent,0.104427
2,nonevent,0.024644
3,nonevent,0.072150
4,nonevent,0.026927
...,...,...
960,event,0.646027
961,nonevent,0.014614
962,event,0.912476
963,nonevent,0.004240


In [405]:
clf.fit(X_train, y_train).score(X_test, y_test)

0.8756476683937824

### Multi-class

Only try to classify the event days since binary classifier already classifies nonevent days fairly accurately

In [406]:
train_data.insert(1, "class4", pd.read_csv("npf_train.csv")["class4"])

train_data_event = train_data[train_data["class4"] != "nonevent"].drop("class2", axis = 1)

In [407]:
from sklearn.ensemble import RandomForestClassifier

X_train_multiclass = X_train[(y_train == 1.0), :]
y_train_multiclass, uniques = pd.factorize(train_data_event["class4"])

X_test_multiclass = X_test[np.array(predictions["class4"] == "event"), :]
y_test_multiclass = np.array(pd.read_csv("npf_test.csv")["class4"])[np.array(predictions["class4"] == "event")]

clf_randomForest = LogisticRegression(penalty = 'l1', C = 1/2, solver = "saga", max_iter = 5000, multi_class = 'multinomial')

scores = cross_validate(
    clf_randomForest, X_train_multiclass, y_train_multiclass, cv = 10, scoring = ('accuracy', 'neg_mean_squared_error')
)

random_forest_predictions = pd.DataFrame(clf_randomForest.fit(X_train_multiclass, y_train_multiclass).predict(X_test_multiclass), columns = ["class4"])
random_forest_predictions = random_forest_predictions["class4"].map({0: 'Ib', 1:'II', 2:'Ia'})
random_forest_predictions.to_csv("multiclass_answers.csv")

scores["test_accuracy"].mean()
scores["test_neg_mean_squared_error"].mean()

-0.7396739130434782

In [385]:
random_forest_predictions


test_df = predictions["class4"]

#Combine the event day predictions to the binary predictions, and calculate the accuracy
j = 0
for i in range(test_df.shape[0]):
    if (test_df[i] == "event"):
        test_df.iloc[i] = random_forest_predictions.iloc[j]
        j += 1

test_df

actual_class4 = pd.read_csv("npf_test.csv")["class4"]

correct = (test_df == actual_class4)

accuracy = correct.sum() / correct.size
accuracy

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


0.7088082901554404

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size = 0.5, random_state = 42
)

